<a href="https://colab.research.google.com/github/EmanElrefai/Islamic-chatbot/blob/master/Islamic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Utilization*

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
import json
import pickle
import tensorflow as tf

import numpy as np
import random
from keras.models import Sequential
from keras.layers import Activation,Dropout,Dense
from keras.models import load_model

#chatbot gui
from termcolor import colored




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
ls

classes.pkl  intents.json  sample_data/  words.pkl


# Process data

In [12]:
words=[]
documents=[]
classes=[]
ignore_words=['?','!','.','“', '”','(', ')', ',','’']

data_file = open('intents.json',)
intents=json.load(data_file)


In [13]:
nltk.download('wordnet')

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)

    documents.append((w,intent['tag']))
    # add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#lemizer and lower each word and remove duplicates
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

#sort classes
classes=sorted(list(set(classes)))

#decuments= pattern , intents
print(len(documents),"documents")
#classes= intents
print(len(classes),"classes",classes)
#words = each words , vocabulary
print(len(words),"unique lemmatized words",words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))




194 documents
108 classes ['', 'NGOs', 'agricultural financing', 'automatic deduction of zakat', 'bankruptcy or dafult', 'bill', 'blockchain', 'bonds', 'challanges', 'challenges', 'charitable activities or charity', 'close account', 'contracts', 'credit cards or debit cards', 'customer disputes', 'cybersecurity', 'dafault account', 'data privacy', 'debt recovery', 'default assets', 'deposit', 'deposits', 'difference between bonds', 'difference islamic banking', 'disable', 'economic development', 'education and health loan', 'education sector', 'endowment', 'ethical guidelines', 'financial instruments', 'foreign currency', 'foreign exchange trading', 'fraud', 'freelancers', 'goals', 'goodbye', 'governed', 'greeting', 'guidelines for investing', 'hajj and umrah', 'haram', 'hawala', 'healthcare industry', 'infrastructure financing', 'inheritance', 'insider trading', 'insurance', 'insurance difference', 'intellectual', 'interest', 'interest free transaction', 'international transactions', 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Training data

In [14]:

#create training data
training=[]

#create output  empty array
output_=[0]*len(classes)


#bag of words algorithm
for doc in documents:
    #intialize bag for each word
    bag=[]

    pattern_word=doc[0]

    #lemmatize and lower each word in pattern_word
    pattern_word=[lemmatizer.lemmatize(word.lower()) for word in pattern_word]

    for w in words:
      bag.append(1) if w in pattern_word else bag.append(0)

    #output
    output_row=list(output_)
    output_row[classes.index(doc[1])]=1

    training.append([bag,output_row])

#shuffel features and turn into array
random.shuffle(training)
training=np.array(training)

#train_x is input and train_y is output
train_x=list(training[:,0])
train_y=list(training[:,1])

print('training data created')



training data created


<ipython-input-14-7fb4f731552e>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [15]:
#building sequential model with 3 layers

model=Sequential()
#first layer with 128 neurons
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
#second layer with 64 neurons
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(len(train_y[0]),activation='softmax'))

# stochastic gradient decsent
sgd=tf.keras.optimizers.legacy.SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
#compiling model
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

#fitting model
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1,validation_split=0.2)
#saving model
model.save('chatbot_model.h5',hist)

print('model created')




Epoch 1/200
31/31 [==============================] - 1s 11ms/step - loss: 4.6999 - accuracy: 0.0129 - val_loss: 4.6880 - val_accuracy: 0.0000e+00
Epoch 2/200
31/31 [==============================] - 0s 4ms/step - loss: 4.6426 - accuracy: 0.0258 - val_loss: 4.6993 - val_accuracy: 0.0000e+00
Epoch 3/200
31/31 [==============================] - 0s 4ms/step - loss: 4.5775 - accuracy: 0.0774 - val_loss: 4.7173 - val_accuracy: 0.0000e+00
Epoch 4/200
31/31 [==============================] - 0s 3ms/step - loss: 4.5535 - accuracy: 0.0581 - val_loss: 4.7249 - val_accuracy: 0.0000e+00
Epoch 5/200
31/31 [==============================] - 0s 3ms/step - loss: 4.4750 - accuracy: 0.0516 - val_loss: 4.7267 - val_accuracy: 0.0000e+00
Epoch 6/200
31/31 [==============================] - 0s 3ms/step - loss: 4.4333 - accuracy: 0.0710 - val_loss: 4.7239 - val_accuracy: 0.0000e+00
Epoch 7/200
31/31 [==============================] - 0s 3ms/step - loss: 4.3723 - accuracy: 0.0774 - val_loss: 4.6992 - val_accur

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#predicting the response

In [16]:
#loading words and classes files

words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))

In [17]:
def clean_up_sentence(sentence):
  #toknize sentence
  sentence_words=nltk.word_tokenize(sentence)
  #lemmtaized and lower each word
  sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

def bow(sentence,words,show_details=False):
  #cleaning up sentence
  sentence_words=clean_up_sentence(sentence)

  bag=[0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        # bag equal one if current word is in the vocabulary position
        bag[i]=1

        if show_details==True:
          print('Found in %s',w)

  return np.array(bag)

In [18]:
def predict_class(sentence,model):
  #filter prediction that below threshold
  p=bow(sentence,words,show_details=False)
  res=model.predict(np.array([p]))[0]

  ERROR_THRESHOLD=0.25

  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  #sort by strong probability
  results.sort(key=lambda x:x[1],reverse=True)

  return_list=[]
  for r in results:
      return_list.append({"intent":classes[r[0]],"probability":str(r[1])})


  return return_list






# Running chatbot

In [19]:
def getResponse(ints,intents_json):
  tag=ints[0]['intent']
  list_intents=intents_json['intents']

  for i in list_intents:

    if (tag==i['tag']):
      result=random.choice(i['responses'])
      break

  return result


def chatbot_response(msg):
  ints=predict_class(msg,model)
  res=getResponse(ints,intents)

  return res



In [20]:
def run():
  print(colored("WELCOME TO ISLAMIC CHATBOT", 'red'))
  print("")
  while(1):
     request=input("")
     response=chatbot_response(request)
     print(colored(response, 'cyan'))
     if request=="goodbye"or request== "Bye" or request=="See you later" or request== "Goodbye" or request=="Nice chatting to you, bye" or request=="Till next time":
           break









In [ ]:
run()

WELCOME TO ISLAMIC CHATBOT

What is interest
1/1 [==============================] - 0s 138ms/step
In Islam, the concept of interest riba is considered Haram prohibited based on the following Quranic verses:Those who consume interest cannot stand on the Day of Resurrection except as one stands who is being beaten by Satan into insanity. That is because they say, Trade is just like interest. But Allah has permitted trade and has forbidden interest. (Quran 2:275)
what is Islam
1/1 [==============================] - 0s 30ms/step
Hawala is a traditional informal remittance system used in Islamic banking and finance. It involves transferring money or value between individuals or entities through a network of intermediaries without physically moving the funds across borders. The system relies on trust and the reputation of the intermediaries, making it a common method for remittances, trade settlements, and cross-border transactions within the Islamic banking context.
